In [3]:
import openpyxl
from copy import copy
from openpyxl.styles import PatternFill

# 元のファイルを読み込む
file_path = 'データ置き場/LB71_IRS23-00445_A052_LB68_IRS23-00669_B027_1_CRC-7BEBDE36_Classified.xlsx'  # 元のファイルのパスを指定
workbook = openpyxl.load_workbook(file_path, data_only=True)

# 処理1: 「変更1」シートのB列のデータを背景色に基づいて並び替える
sheet = workbook['変更1']
data_with_colors = []
for row in range(7, 152):  # B列の7行目から151行目まで
    cell = sheet.cell(row, 2)  # B列
    data_with_colors.append((cell.value, cell.fill.start_color.rgb if cell.fill.fill_type else None))

data_with_colors.sort(key=lambda x: x[1] or '')  # 背景色で並び替え

# 新しいワークブックを作成し、「並び替え」シートにデータを出力
new_workbook = openpyxl.Workbook()
sort_sheet = new_workbook.create_sheet('定数並び替え一覧', 0)
for index, (value, color) in enumerate(data_with_colors, start=7):
    cell = sort_sheet.cell(row=index, column=2)  # B列
    cell.value = value
    if color:
        cell.fill = PatternFill(start_color=color, fill_type="solid")
        
# 処理2: 「変更1」シートのA列からR列のデータと色情報を「仮」シートにコピー
temp_sheet = new_workbook.create_sheet('生データ', 1)
for row in range(1, sheet.max_row + 1):
    for col in range(1, 19):  # A列(1)からR列(18)まで
        original_cell = sheet.cell(row, col)
        new_cell = temp_sheet.cell(row, col)
        new_cell.value = original_cell.value
        if original_cell.has_style:  # セルにスタイルがあればコピー
            new_cell.font = copy(original_cell.font)
            new_cell.border = copy(original_cell.border)
            new_cell.fill = copy(original_cell.fill)
            new_cell.number_format = copy(original_cell.number_format)
            new_cell.protection = copy(original_cell.protection)
            new_cell.alignment = copy(original_cell.alignment)

# 処理3: 「仮」シートのデータを条件に基づいて「生データ_抽出」シートにコピー
max_row_temp_sheet = temp_sheet.max_row
temp2_sheet = new_workbook.create_sheet('生データ_抽出', 2)
for row in range(1, max_row_temp_sheet + 1):
    if row + 2 <= max_row_temp_sheet and temp_sheet.cell(row + 2, 1).value == '変更理由':
        temp2_sheet.cell(row, 1).value = temp_sheet.cell(row, 1).value
        
# 処理4: 「生データ_抽出」シートのA列データの隣の列に行数を追記
max_row_temp2_sheet = temp2_sheet.max_row
for row in range(1, max_row_temp2_sheet + 1):
    current_cell = temp2_sheet.cell(row, 1)
    if current_cell.value is not None:
        count = 0
        for next_row in range(row + 1, max_row_temp2_sheet + 1):
            if temp2_sheet.cell(next_row, 1).value is not None:
                break
            count += 1
        temp2_sheet.cell(row, 2).value = count

# 処理5: 「定数並び替え一覧」と「生データ_抽出」シートを比較し、「物理値並び替え一覧」シートに一致するデータを追記
temp3_sheet = new_workbook.create_sheet('物理値並び替え一覧')
sort_data = [cell.value for cell in sort_sheet['B'][6:151]]  # B7 to B151 in '定数並び替え一覧' sheet
temp2_data = [cell.value for cell in temp2_sheet['A'][:max_row_temp2_sheet]]  # Up to A3883 in '生データ_抽出' sheet

output_row_index = 221  # Start output row
for data in sort_data:
    if data in temp2_data:
        # Output matching data to '物理値並び替え一覧' sheet
        temp3_sheet.cell(row=output_row_index, column=1).value = data

        # Retrieve the count of matching data rows from column B of '生データ_抽出' sheet
        match_row_index = temp2_data.index(data)
        space_count = temp2_sheet.cell(row=match_row_index + 1, column=2).value or 0
        
        temp3_sheet.cell(row=output_row_index, column=2).value = space_count

        # Update the row index for the next matching data
        output_row_index += space_count + 1  # Add the count of blank rows and the current data row

# Saving the final workbook
final_workbook_path = '中間成果物.xlsx'
new_workbook.save(final_workbook_path)
final_workbook_path

'中間成果物.xlsx'

In [22]:
# Adjusting the code to compare A column values row by row from row 22 onwards in both sheets
# and then output matching strings in the new sheet at the same row as in '物理値並び替え一覧'
# Also, adding logging to display which rows are being compared and matched

# 元のファイルを読み込む
workbook = openpyxl.load_workbook(final_workbook_path)

# Adjusting the code to:
# 1. Find rows in '物理値並び替え一覧' where A column has a string
# 2. Compare these values with all rows in A column of '生データ'
# 3. If a match is found, output the matching string to the new file at the corresponding row in A column
# 4. Output the values of the next row (y+1) from '生データ' in columns A and B to the next row (x+1) in columns A and B of the new file

# Function to copy cell properties
def copy_cell_properties(source_cell, target_cell):
    target_cell.value = source_cell.value
    target_cell.font = Font(name=source_cell.font.name, size=source_cell.font.size,
                            bold=source_cell.font.bold, italic=source_cell.font.italic,
                            vertAlign=source_cell.font.vertAlign, underline=source_cell.font.underline,
                            strike=source_cell.font.strike, color=source_cell.font.color)
    target_cell.border = Border(left=source_cell.border.left, right=source_cell.border.right,
                                top=source_cell.border.top, bottom=source_cell.border.bottom,
                                diagonal=source_cell.border.diagonal, diagonal_direction=source_cell.border.diagonal_direction,
                                outline=source_cell.border.outline, vertical=source_cell.border.vertical,
                                horizontal=source_cell.border.horizontal)
    target_cell.fill = PatternFill(fill_type=source_cell.fill.fill_type, start_color=source_cell.fill.start_color,
                                   end_color=source_cell.fill.end_color)
    target_cell.number_format = source_cell.number_format
    target_cell.protection = Protection(locked=source_cell.protection.locked, hidden=source_cell.protection.hidden)
    target_cell.alignment = Alignment(horizontal=source_cell.alignment.horizontal, vertical=source_cell.alignment.vertical,
                                      text_rotation=source_cell.alignment.text_rotation, wrap_text=source_cell.alignment.wrap_text,
                                      shrink_to_fit=source_cell.alignment.shrink_to_fit, indent=source_cell.alignment.indent)

# Initializing the new workbook and sheet for output
new_workbook = openpyxl.Workbook()
new_sheet = new_workbook.active
new_sheet.title = '物理値並び替え一覧'

# Access the two sheets again
sheet_raw_data = workbook['生データ']
sheet_sorted_values = workbook['物理値並び替え一覧']

# Iterate over '物理値並び替え一覧' and compare with '生データ'
for row, cell in enumerate(sheet_sorted_values['A'], start=1):
    if cell.value:  # Check if there is a string in the cell
        for raw_row, raw_cell in enumerate(sheet_raw_data['A'], start=1):
            if cell.value == raw_cell.value:
                # Output the matching string in column A
                #new_sheet.cell(row=row, column=1, value=cell.value)
                # Output the value from '物理値並び替え一覧' B column to the new file B column
                #new_sheet.cell(row=row, column=2, value=sheet_sorted_values.cell(row=row, column=2).value)
                
                # Output the matching string and copy cell properties in column A
                copy_cell_properties(cell, new_sheet.cell(row=row, column=1))
                # Output the value and copy cell properties from '物理値並び替え一覧' B column
                copy_cell_properties(sheet_sorted_values.cell(row=row, column=2), new_sheet.cell(row=row, column=2))

                # Output the values from the next row of '生データ' in columns A and B to the next row in columns A and B of new file
                if raw_row < sheet_raw_data.max_row:
                    for gyo in range(sheet_sorted_values.cell(row=row, column=2).value):
                        gyo = gyo + 1
                        for retsu in range(18):#R
                            retsu = retsu + 1
                            next_raw_value = sheet_raw_data.cell(row=raw_row + gyo, column=retsu).value
                            new_sheet.cell(row=row + gyo, column=retsu, value=next_raw_value)
                break


# Save the updated new workbook
updated_output_file_path_advanced_b = '変更後_一致した文字列_拡張版_B列.xlsx'
new_workbook.save(updated_output_file_path_advanced_b)

updated_output_file_path_advanced_b


NameError: name 'Font' is not defined

In [17]:
sheet_sorted_values.cell(row, 1)

<Cell '物理値並び替え一覧'.A162>

In [18]:
sheet_raw_data.max_row

3883

In [10]:
sheet_sorted_values

<Worksheet "物理値並び替え一覧">

In [11]:
sheet_raw_data

<Worksheet "生データ">

In [32]:
# I will fix the NameError by defining the Font import properly
from openpyxl.styles import Font, PatternFill, Border, Alignment

# Define the function to copy cell properties again with the correct imports
def copy_cell_properties(source_cell, target_cell):
    # Copy value
    target_cell.value = source_cell.value

    # Copy font properties
    if source_cell.font:
        target_cell.font = Font(name=source_cell.font.name, 
                                size=source_cell.font.size,
                                bold=source_cell.font.bold, 
                                italic=source_cell.font.italic,
                                vertAlign=source_cell.font.vertAlign, 
                                underline=source_cell.font.underline,
                                strike=source_cell.font.strike, 
                                color=source_cell.font.color)

    # Copy fill properties
    if source_cell.fill:
        target_cell.fill = PatternFill(fill_type=source_cell.fill.fill_type,
                                       start_color=source_cell.fill.start_color,
                                       end_color=source_cell.fill.end_color)

    # Copy border properties
    if source_cell.border:
        target_cell.border = Border(left=source_cell.border.left, 
                                    right=source_cell.border.right,
                                    top=source_cell.border.top, 
                                    bottom=source_cell.border.bottom,
                                    diagonal=source_cell.border.diagonal,
                                    diagonal_direction=source_cell.border.diagonal_direction,
                                    outline=source_cell.border.outline,
                                    vertical=source_cell.border.vertical,
                                    horizontal=source_cell.border.horizontal)

    # Copy alignment properties
    if source_cell.alignment:
        target_cell.alignment = Alignment(horizontal=source_cell.alignment.horizontal,
                                          vertical=source_cell.alignment.vertical,
                                          text_rotation=source_cell.alignment.text_rotation,
                                          wrap_text=source_cell.alignment.wrap_text,
                                          shrink_to_fit=source_cell.alignment.shrink_to_fit,
                                          indent=source_cell.alignment.indent)

# Iterate over '物理値並び替え一覧' and compare with '生データ'
for row, cell in enumerate(sheet_sorted_values['A'], start=1):
    if cell.value:  # Check if there is a string in the cell
        for raw_row, raw_cell in enumerate(sheet_raw_data['A'], start=1):
            if cell.value == raw_cell.value:
                # Output the matching string and copy cell properties in column A
                copy_cell_properties(cell, new_sheet.cell(row=row, column=1))
                # Output the value and copy cell properties from '物理値並び替え一覧' B column
                copy_cell_properties(sheet_sorted_values.cell(row=row, column=2), new_sheet.cell(row=row, column=2))
                
                # Output the value from '物理値並び替え一覧' B column to the new file B column
                #new_sheet.cell(row=row, column=2, value=sheet_sorted_values.cell(row=row, column=2).value)

                # Output the values and copy cell properties from the next row of '生データ' in columns A and B
                if raw_row < sheet_raw_data.max_row:
                    for gyo in range(sheet_sorted_values.cell(row=row, column=2).value):
                        gyo = gyo + 1
                        for retsu in range(18):
                            retsu = retsu + 1
                            raw_next_cell = sheet_raw_data.cell(row=raw_row + gyo, column=retsu)
                            new_next_cell = new_sheet.cell(row=row + gyo, column=retsu)
                            copy_cell_properties(raw_next_cell, new_next_cell)
                break


# Save the updated new workbook
updated_output_file_path_advanced_b = '変更後_一致した文字列_拡張版_B列.xlsx'
new_workbook.save(updated_output_file_path_advanced_b)

updated_output_file_path_advanced_b                

'変更後_一致した文字列_拡張版_B列.xlsx'